# Mine releases from selected projects

This notebook mine the releases

In [1]:
import os
import datetime
import json
import re
import traceback
import math

from multiprocessing import Pool
import pandas as pd

from ipywidgets import IntProgress
from IPython.display import display

import releasy

In [2]:
from util import (
    DATA_PATH,
    REPO_PATH,
    TMP_PATH, 
    CPU,
    CycleType,
    RAPID_RELEASE_LIM,
    TRAD_RELEASE_LIM,
    delta2days,
    is_rapid_release,
    is_trad_release,
    get_cycle,
    get_duration,
    get_start_delay,
    get_commits_in_advance
)

In [3]:
selected_projects = pd.read_csv(DATA_PATH / '10_projects_selected.csv')

## Mine Releases


In [4]:
def mine(name: str) -> releasy.Project:
    elapsed_time = datetime.datetime.now()
    
    try:
        repo_path = str(REPO_PATH / name)
        project = releasy.Miner(repo_path, name).apply(
            releasy.FinalReleaseMiner(),
            releasy.HistoryCommitMiner(),
            releasy.BaseReleaseMiner(),
            releasy.ContributorMiner(),
            releasy.SemanticReleaseMiner()
        ).mine()

        rapid_releases = [release for release in project.main_releases if is_rapid_release(release)]
        trad_releases = [release for release in project.main_releases if is_trad_release(release)]

        project_data = {
            'project': project.name,
            'prefixes': len(project.releases.prefixes()),
            'prefixes_names': str(" ".join(project.releases.prefixes())),
            'main_releases': len(project.main_releases),
            'rapid_releases': len(rapid_releases),
            'trad_releases': len(trad_releases),
            'patches': len(project.patches),
            'releases': len(project.releases)
        }
        
        release_data = []
        for srelease in project.main_releases:
            # release = srelease.release
            # cycle_days = delta2days(get_cycle(srelease))
            # delay = delta2days(srelease.delay)
            # prev_release = srelease.prev_main_release

            if not srelease.commits or not srelease.prev_main_release:
                continue
                
            cycle_days = delta2days(get_cycle(srelease))
            duration = delta2days(get_duration(srelease))
            start_delay = delta2days(get_start_delay(srelease))
            commits_in_advance = get_commits_in_advance(srelease)
            
            release_data.append({
                'project': project.name,
                'release': srelease.name,
                'previous_release': srelease.prev_main_release.name if srelease.prev_main_release else '',
                'cycle_old': delta2days(srelease.cycle),
                'cycle_days': cycle_days,
                'rapid_release': is_rapid_release(srelease),
                'trad_release': is_trad_release(srelease),
                
                'commits': len(srelease.commits),
                'commits_in_advance': len(commits_in_advance),
                'start_delay': start_delay,
                'duration': duration,
                'patches': len(srelease.patches)     
            })
        
        elapsed_time = datetime.datetime.now() - elapsed_time
        project_data['time'] = elapsed_time
    except Exception as err:
        print(f"{name:40} {err=}")
        traceback.print_exception(err)
        project_data = {}
        release_data = []
    
    return project_data, release_data

In [5]:
project_names = list(selected_projects['project'])
# project_names = project_names[:1]

In [6]:
with Pool(processes=CPU) as pool:
    processed = 0
    project_results = []
    release_results = []
    progress = IntProgress(min=0, max=len(project_names))
    display(progress)
    for result in pool.imap_unordered(mine, project_names):
        project_result, release_result = result
        project_results.append(project_result)
        release_results.extend(release_result)
        progress.value += 1


IntProgress(value=0, max=1309)

In [7]:
projects = pd.DataFrame(project_results)
projects.sample(1)

,project,prefixes,prefixes_names,main_releases,rapid_releases,trad_releases,patches,releases,time
323,react-grid-layout/react-draggable,1,v,24,9,7,57,81,0 days 00:00:00.052262


In [8]:
releases = pd.DataFrame(release_results)
releases.sample(10)

,project,release,previous_release,cycle_old,cycle_days,rapid_release,trad_release,commits,commits_in_advance,start_delay,duration,patches
8823,prometheus/node_exporter,v0.13.0,0.12.0,204.622396,204.622928,False,True,98,1,-51.381435,256.004363,0
31572,wekan/wekan,v4.07,v4.06,0.052708,0.052708,True,False,8,1,-0.056065,0.108773,0
2296,nhn/tui.image-editor,v3.15.0,v3.14.0,141.811748,141.811748,False,True,12,0,43.740220,98.071528,3
7298,HMBSbige/ShadowsocksR-Windows,2.5,2.4,13.981146,13.981146,True,False,12,0,0.007836,13.973310,9
4311,aws/aws-sdk-php,3.12.0,3.11.0,32.060243,32.060243,True,False,11,0,27.457674,4.602569,2
42563,laravel/framework,v8.13.0,v8.12.0,5.046667,5.046667,True,False,31,0,0.992836,4.053831,0
5570,software-mansion/react-native-reanimated,1.8.0,1.7.0,81.870370,81.870370,False,False,14,0,67.711586,14.158785,0
39722,rclone/rclone,v1.35,v1.34,57.203079,57.203079,False,False,79,7,-14.982581,72.185660,0
10460,wix/react-native-calendars,1.195.0,1.194.0,3.857049,3.857095,True,False,2,0,3.855961,0.001134,0
2395,hiroppy/fusuma,v1.15.0,v1.14.0,40.253137,40.253125,True,False,10,0,13.708981,26.544144,2


In [9]:
len(projects)

1309

In [10]:
len(releases)

42675

In [11]:
len(releases.query("rapid_release == True"))

26040

In [12]:
len(releases.query("trad_release == True"))

9799

In [13]:
len(releases.query("rapid_release == False and trad_release == False"))

6836

In [14]:
releases['commits'].sum()

4210571

In [15]:
projects.to_csv(DATA_PATH / '20_projects.csv', index=False)

In [16]:
releases.to_csv(DATA_PATH / '20_releases.csv', index=False)

In [17]:
releases.query('abs(cycle_old - cycle_days) > 100')

,project,release,previous_release,cycle_old,cycle_days,rapid_release,trad_release,commits,commits_in_advance,start_delay,duration,patches
1020,express-validator/express-validator,v0.3.0,v0.1.0,1301.377454,369.069294,False,True,4,0,352.978009,16.091285,2
1021,express-validator/express-validator,v0.4.0,v0.3.0,0.000440,241.228576,False,True,11,0,182.166644,59.061933,1
1027,express-validator/express-validator,v2.8.0,v2.7.0,166.397963,40.389549,True,False,2,0,38.761539,1.628009,0
1201,jrnl-org/jrnl,v2.0,v1.9.0,1728.455104,2090.541493,False,True,175,14,-26.360058,2116.901551,2
1202,jrnl-org/jrnl,v2.2,v2.0,293.524606,0.000012,True,False,53,53,-160.740162,160.740174,0
...,...,...,...,...,...,...,...,...,...,...,...,...
40534,fxsjy/jieba,v0.16,v0.3,0.016597,-265.200579,False,False,1,1,-265.200579,0.000000,0
40956,mitmproxy/mitmproxy,v2.0.0,v1.0,377.857523,57.793102,False,False,239,0,0.915081,56.878021,2
40957,mitmproxy/mitmproxy,v3.0.0,v2.0.0,45.704155,365.765301,False,True,1238,5,-8.852164,374.617465,4
41669,SortableJS/Sortable,1.7.0,1.6.0,586.593889,157.110069,False,True,38,1,-93.761192,250.871262,0


In [18]:
releases.query('(cycle_old - cycle_days) < -100')

,project,release,previous_release,cycle_old,cycle_days,rapid_release,trad_release,commits,commits_in_advance,start_delay,duration,patches
1021,express-validator/express-validator,v0.4.0,v0.3.0,0.000440,241.228576,False,True,11,0,182.166644,59.061933,1
1201,jrnl-org/jrnl,v2.0,v1.9.0,1728.455104,2090.541493,False,True,175,14,-26.360058,2116.901551,2
1472,kaorun343/vue-property-decorator,v7.3.0,v7.2.0,0.000694,101.332685,False,True,14,0,0.004375,101.328310,0
2616,reek/anti-adblock-killer,v7.7,v7.6,0.001620,119.889942,False,True,44,0,0.838206,119.051736,0
2795,journeyapps/zxing-android-embedded,v4.3.0,v4.2.0,0.040718,223.922662,False,True,15,0,223.801933,0.120729,0
4992,baldurk/renderdoc,v0.21,v0.20,29.788854,145.216991,False,True,417,0,4.472512,140.744479,0
6117,thedevs-network/kutt,v2.4.0,v2.3.0,66.883634,171.679757,False,True,3,0,169.626516,2.053241,6
7901,mrdoob/stats.js,r5,r4,0.001296,103.659792,False,True,8,0,0.001007,103.658785,0
8583,PrestaShop/PrestaShop,1.5.0.1,1.5.0.0,0.000000,140.343785,False,True,1724,0,0.040718,140.303067,0
8818,prometheus/node_exporter,0.8.0,0.7.0,0.002257,102.857292,False,True,44,0,20.845440,82.011852,1
